## ЗАДАНИЕ ДЛЯ ПРОГРАММИРУЮЩИХ

**Ссылка**, на источник текста

In [22]:
# DATA_URL = "http://az.lib.ru/t/tolstoj_a_k/text_0180.shtml" Оригинальный текст. Все вычеркнутые значения относятся к этому тексту
DATA_URL = "http://az.lib.ru/t/tolstoj_a_k/text_0170.shtml"

Устанавливаем библиотеки

In [23]:
import warnings
warnings.filterwarnings('ignore')
!pip install -q tensorflow==2.12
!pip install -q git+https://github.com/dvolchek/rnnmorph.git

  Preparing metadata (setup.py) ... done


Создаём объект морфологического анализатора `RNNMorph`

In [24]:
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")

Скачиваем текст, по которому будет дано задание, с помощью `urllib`

In [25]:
import urllib.request

opener = urllib.request.URLopener({})
resource = opener.open(DATA_URL)
raw_text = resource.read().decode(resource.headers.get_content_charset()) #Текс с html тегами

In [26]:
raw_text[:200]

'<html>\r\n<head>\r\n<title>Lib.ru/Классика: Толстой Алексей Константинович. Упырь</title>\r\n</head>\r\n\r\n<body>\r\n\r\n\r\n<center>\r\n\r\n<h2><a href=/t/tolstoj_a_k/>Толстой Алексей Константинович</a><br>\r\nУпырь</h2>'

Как видно, текст содержит html теги, от которых нужно избавиться. Выбрасываем из текста HTML-теги с помощью библиотеки Beatiful soap

In [27]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_text, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
cleaned_text = soup.get_text()

In [28]:
cleaned_text[:200]

'\n\nLib.ru/Классика: Толстой Алексей Константинович. Упырь\n\n\n\nТолстой Алексей Константинович\r\nУпырь\n\n\nLib.ru/Классика:\n\r\n\n\n[Регистрация]\n \n\r\n\r\n\r\n[Найти] \r\n[Рейтинги]\r\n[Обсуждения]\r\n[Новинки]\r\n[Обзоры]\r\n'

С помощью библиотеки [NLTK](https://nltk.org/) разбиваем текст на предложения и токены.

In [29]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

tokenized_sentences = [word_tokenize(sentence, language="russian") for sentence in sent_tokenize(cleaned_text)]
"A total of %d 'sentences'" % len(tokenized_sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


"A total of 1464 'sentences'"

## Задание 1
С помощью метода `str.isalpha` из стандартной библиотеки Python модифицируйте нижеследующий код так, чтобы в predictions остались только буквенные токены.

In [37]:
from tqdm import tqdm
predictions = [[pred.normal_form for pred in sent if pred.normal_form.isalnum()]
               for sent in tqdm(predictor.predict_sentences(sentences=tokenized_sentences), "sentences") ]
predictions[-11:0] #Сейчас видно, что токены типа "точка", "запятая" и тд пока присутствуют в предложениях. От них нужно избавиться



23/23 [==============================] - 13s 567ms/step


sentences: 100%|██████████| 1464/1464 [00:00<00:00, 78771.32it/s]


[]

Проверьте себя. Должны получиться следующие значения:

*   Предложений: ~577~ 1464 (возможны расхождения в несколько предложений)
*   Токенов: примерно ~8621~ 20881 (возможны расхождения в некоторое количество токенов)

In [38]:
len(predictions)

1464

In [39]:
non_uniq_tokens = [word for sentence in predictions for word in sentence]
len(non_uniq_tokens)


20881

Для продолжения работы над заданием числа должны быть близки к указанным

## Задание 2

Используя `non_uniq_tokens`, стоп-слова для русского языка из библиотеки nltk (`nltk.corpus.stopwords`) и `nltk.FreqDist`, вычислите, **какую долю среди ~~100~~ 40 самых частотных** токенов в произведении занимают токены, **не относящиеся** к стоп словам.

**Например**, если среди 100 самых частотных слов встречается 25 слов, входящих в стоп лист, значит не входят в стоп лист 75 слов, и их доля составит 0.75.

**Не бойтесь использовать документацию NLTK и тьюториалы.**

In [40]:
import nltk
from nltk import FreqDist
from nltk.corpus import stopwords
nltk.download("stopwords")
STOPWORDS = set(stopwords.words("russian"))
stopwords.words("russian")[:5] #Пример стоп слов

freq_dist = FreqDist(non_uniq_tokens)

top_40_tokens = [token for token, _ in freq_dist.most_common(40)]

non_stop_tokens = [token for token in top_40_tokens if token not in STOPWORDS]

non_stop_ratio = len(non_stop_tokens) / 100
non_stop_ratio

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0.1

Проверьте себя: должно получиться ~0.49~ 0.19 (допустимо небольшое расхождение)

## Задание 3
Вычислите, сколько токенов встречается в тексте **строго больше** ~~50~~ 10 раз.

In [41]:
tokens_more_than_10 = [token for token, count in freq_dist.items() if count > 10]
num_tokens_more_than_10 = len(tokens_more_than_10)
num_tokens_more_than_10

276

Проверьте себя: должно получиться значение ~~22~~ 276 (возможно небольшое расхождение)
